In [ ]:
# https://www.youtube.com/watch?v=Jfpxjg8xj9w

In [ ]:
!pip install -q dspy-ai lamini rich

In [ ]:
import sys
import os
import dspy
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate.evaluate import Evaluate
from dsp.utils import deduplicate
from rich import print
from dsp import LM
import lamini
from dspy.datasets import HotPotQA
import dspy
from dspy.evaluate.evaluate import Evaluate
# from dspy.teleprompt import bootstrapFewShot

In [ ]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# Set up the LM.
turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)
dspy.settings.configure(lm=turbo)

# Load math questions from the GSM8K dataset.
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

100%|██████████| 1319/1319 [00:00<00:00, 18235.20it/s]


In [ ]:
!pip install -q dspy-ai

In [ ]:
import dspy
import time
class CustomLMClient(LM):
    def __init__(self):
        self.provider = "lamini"
        self.history = []
        lamini.api_key = "030f9952a4ee130a33d4fcfd7a7f3dbeab6b2a24a46917b6681019267130e468"
        self.llm = lamini.Lamini("mistralai/Mistral-7B-Instruct-v0.3")
        # Add a kwargs attribute to store keyword arguments
        self.kwargs = {'temperature' : 0.7 , 'max_tokens': 100}

    def basic_request(self, prompt, **kwargs):
        # Call the lamini API to generate a response
        print("$ API Request sent")
        response = self.llm.generate(prompt)
        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        # Add the prompt to the history
        self.history.append(prompt)

        # Get the response using the basic_request method
        response = self.basic_request(prompt)

        # Return the response
        return response

custom_lm = CustomLMClient()

# Configure dspy to use your custom LLM
dspy.settings.configure(lm=custom_lm)

In [ ]:
class ShakespeareanTranslation(dspy.Signature):
    """Translate simple english to Shakespearean english."""
    simple_english = dspy.InputField()
    # print(simple_english)
    shakespearean_english = dspy.OutputField()
    # print(shakespearean_english)

In [ ]:
# from google.colab import userdata
# OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
# turbo = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=1000, api_key=OPENAI_API_KEY)
dspy.settings.configure(lm=custom_lm)
from dspy.signatures.signature import signature_to_template
shakespeare_translation_as_template = signature_to_template(ShakespeareanTranslation)

In [ ]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(ShakespeareanTranslation)

    def forward(self, simple_english):
        return self.prog(simple_english=simple_english)
c = CoT()

In [ ]:
c.forward("You should relax and have fun while it lasts")

$ API Request sent

$ API Request sent

Prediction(
    rationale='e',
    shakespearean_english='E',
    completions=Completions(...)
) (2547 completions omitted)